In [3]:
%matplotlib inline
import csv
import numpy as np
from scipy.signal import *
from numpy.fft import *
import matplotlib.pyplot as plt
from ipywidgets import * # Pour insérer des widgets (éléments graphiques) dans le notebook
# Attention! Installer scikit-learn pour avoir accès aux algorithmes de classification
# Installer version 0.20 au plus récente (0.22.1) pour avoir accès à la nouvelle fonction tree.plot_tree()
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier # Méthode KNN
from pandas import * # Structures de données
from sklearn.naive_bayes import GaussianNB
from scipy.ndimage import interpolation
from scipy.stats import entropy

In [4]:
feat_df = DataFrame(columns=['step type','Gyro0X mean', 'Gyro0Y mean', 'Gyro0Z mean',
                                         'Accel0X mean','Accel0Y mean','Accel0Z mean',
                                         'Gyro1X mean', 'Gyro1Y mean','Gyro1Z mean',
                                         'Accel1X mean', 'Accel1Y mean','Accel1Z mean',
                             
                                          'Gyro0X std', 'Gyro0Y std','Gyro0Z std',
                                         'Accel0X std', 'Accel0Y std', 'Accel0Z std',
                                         'Gyro1X std', 'Gyro1Y std', 'Gyro1Z std', 
                                         'Accel1X std', 'Accel1Y std', 'Accel1Z std',
                             
                                         'Gyro0X ent', 'Gyro0Y ent', 'Gyro0Z ent', 
                                         'Accel0X ent', 'Accel0Y ent', 'Accel0Z ent', 
                                         'Gyro1X ent', 'Gyro1Y ent','Gyro1Z ent',
                                         'Accel1X ent','Accel1Y ent','Accel1Z ent',
                             
                                         'Gyro0X corr', 'Gyro0Y corr', 'Gyro0Z corr', 
                                         'Accel0X corr', 'Accel0Y corr', 'Accel0Z corr', 
                                         'Gyro1X corr', 'Gyro1Y corr','Gyro1Z corr',
                                         'Accel1X corr','Accel1Y corr','Accel1Z corr'])

In [5]:
step_type_array = ["brush_back","brush_forward","dig","heel","step"]
size_array = [25,25,23,21,22]

for i in range(len(size_array)):
    step_type = step_type_array[i]
    size = size_array[i]
    
    for k in range(1):
        if k == 0:
            path_prep =  "etapdance2" + "/" + step_type + "/" + step_type + "_prep" + "/" + step_type + "_prep_"
        elif k == 1:
            path_prep =  "etapdance2" + "/" + step_type + "/" + step_type + "_filt" + "/" + step_type + "_filt_"

        for n in range(size+1):
            step_type_nb = str(n)
            ext = ".txt"
            data_input = path_prep + step_type_nb + ext

            data_prep = pandas.read_csv(data_input,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 
            
            # pour calcul corrélation
            step_corr = "step"
            data_input_corr = "etapdance2" + "/" + step_corr + "/" + step_corr + "_prep_mean" + ext
            data_int_mean = pandas.read_csv(data_input_corr,names=['Gyro0X (deg)','Gyro0Y (deg)','Gyro0Z (deg)','Accel0X (g)','Accel0Y (g)','Accel0Z (g)','Gyro1X (deg)','Gyro1Y (deg)','Gyro1Z (deg)','Accel1X (g)','Accel1Y (g)','Accel1Z (g)'], dtype='a') 


            # MPU0 6050 (talon)
            Gyro0X = np.array(data_prep['Gyro0X (deg)'])[1:].astype(float)
            Gyro0Y = np.array(data_prep['Gyro0Y (deg)'])[1:].astype(float)
            Gyro0Z = np.array(data_prep['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel0X = np.array(data_prep['Accel0X (g)'])[1:].astype(float)
            Accel0Y = np.array(data_prep['Accel0Y (g)'])[1:].astype(float)
            Accel0Z = np.array(data_prep['Accel0Z (g)'])[1:].astype(float)
            
            # geste moyen MPU0 6050 (talon) pour corrélation
            Gyro0X_int_mean = np.array(data_int_mean['Gyro0X (deg)'])[1:].astype(float)
            Gyro0Y_int_mean = np.array(data_int_mean['Gyro0Y (deg)'])[1:].astype(float)
            Gyro0Z_int_mean = np.array(data_int_mean['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel0X_int_mean = np.array(data_int_mean['Accel0X (g)'])[1:].astype(float)
            Accel0Y_int_mean = np.array(data_int_mean['Accel0Y (g)'])[1:].astype(float)
            Accel0Z_int_mean = np.array(data_int_mean['Accel0Z (g)'])[1:].astype(float)


            # MPU1 6050 (pointe)
            Gyro1X = np.array(data_prep['Gyro1X (deg)'])[1:].astype(float)
            Gyro1Y = np.array(data_prep['Gyro1Y (deg)'])[1:].astype(float)
            Gyro1Z = np.array(data_prep['Gyro1Z (deg)'])[1:].astype(float)

            Accel1X = np.array(data_prep['Accel1X (g)'])[1:].astype(float)
            Accel1Y = np.array(data_prep['Accel1Y (g)'])[1:].astype(float)
            Accel1Z = np.array(data_prep['Accel1Z (g)'])[1:].astype(float)
            
            
            # geste moyen MPU1 6050 (pointe) pour corrélation
            Gyro1X_int_mean = np.array(data_int_mean['Gyro0X (deg)'])[1:].astype(float)
            Gyro1Y_int_mean = np.array(data_int_mean['Gyro0Y (deg)'])[1:].astype(float)
            Gyro1Z_int_mean = np.array(data_int_mean['Gyro0Z (deg)'])[1:].astype(float)
            
            Accel1X_int_mean = np.array(data_int_mean['Accel0X (g)'])[1:].astype(float)
            Accel1Y_int_mean = np.array(data_int_mean['Accel0Y (g)'])[1:].astype(float)
            Accel1Z_int_mean = np.array(data_int_mean['Accel0Z (g)'])[1:].astype(float)
            

            array_feat = [[step_type,   np.mean(Gyro0X),np.mean(Gyro0Y),np.mean(Gyro0Z),
                                        np.mean(Accel0X),np.mean(Accel0Y),np.mean(Accel0Z),
                                        np.mean(Gyro1X),np.mean(Gyro1Y),np.mean(Gyro1Z),
                                        np.mean(Accel1X),np.mean(Accel1Y),np.mean(Accel1Z),

                                        np.std(Gyro0X),np.std(Gyro0Y),np.std(Gyro0Z),
                                        np.std(Accel0X),np.std(Accel0Y),np.std(Accel0Z),
                                        np.std(Gyro1X),np.std(Gyro1Y),np.std(Gyro1Z),
                                        np.std(Accel1X),np.std(Accel1Y),np.std(Accel1Z),

                                        entropy(abs(Gyro0X)),entropy(abs(Gyro0Y)),entropy(abs(Gyro0Z)),
                                        entropy(abs(Accel0X)),entropy(abs(Accel0Y)),entropy(abs(Accel0Z)),
                                        entropy(abs(Gyro1X)),entropy(abs(Gyro1Y)),entropy(abs(Gyro1Z)),
                                        entropy(abs(Accel1X)),entropy(abs(Accel1Y)),entropy(abs(Accel1Z)),
                                        
                                       pandas.Series(Gyro0X_int_mean).corr(pandas.Series(Gyro0X)),
                                       pandas.Series(Gyro0Y_int_mean).corr(pandas.Series(Gyro0Y)),
                                       pandas.Series(Gyro0Z_int_mean).corr(pandas.Series(Gyro0Z)),
                                       pandas.Series(Accel0X_int_mean).corr(pandas.Series(Accel0X)),
                                       pandas.Series(Accel0Y_int_mean).corr(pandas.Series(Accel0Y)),
                                       pandas.Series(Accel0Z_int_mean).corr(pandas.Series(Accel0Z)),

                                       pandas.Series(Gyro1X_int_mean).corr(pandas.Series(Gyro1X)),
                                       pandas.Series(Gyro1Y_int_mean).corr(pandas.Series(Gyro1Y)),
                                       pandas.Series(Gyro1Z_int_mean).corr(pandas.Series(Gyro1Z)),
                                       pandas.Series(Accel1X_int_mean).corr(pandas.Series(Accel1X)),
                                       pandas.Series(Accel1Y_int_mean).corr(pandas.Series(Accel1Y)),
                                       pandas.Series(Accel1Z_int_mean).corr(pandas.Series(Accel1Z))]]


            feat_df_temp = DataFrame(array_feat, columns=['step type','Gyro0X mean', 'Gyro0Y mean', 'Gyro0Z mean',
                                                 'Accel0X mean','Accel0Y mean','Accel0Z mean',
                                                 'Gyro1X mean', 'Gyro1Y mean','Gyro1Z mean',
                                                 'Accel1X mean', 'Accel1Y mean','Accel1Z mean',

                                                 'Gyro0X std', 'Gyro0Y std','Gyro0Z std',
                                                 'Accel0X std', 'Accel0Y std', 'Accel0Z std',
                                                 'Gyro1X std', 'Gyro1Y std', 'Gyro1Z std', 
                                                 'Accel1X std', 'Accel1Y std', 'Accel1Z std',

                                                 'Gyro0X ent', 'Gyro0Y ent', 'Gyro0Z ent', 
                                                 'Accel0X ent', 'Accel0Y ent', 'Accel0Z ent', 
                                                 'Gyro1X ent', 'Gyro1Y ent','Gyro1Z ent',
                                                 'Accel1X ent','Accel1Y ent','Accel1Z ent',
                                                          
                                                 'Gyro0X corr', 'Gyro0Y corr', 'Gyro0Z corr', 
                                                 'Accel0X corr', 'Accel0Y corr', 'Accel0Z corr', 
                                                 'Gyro1X corr', 'Gyro1Y corr','Gyro1Z corr',
                                                 'Accel1X corr','Accel1Y corr','Accel1Z corr'])

            feat_df = feat_df.append(feat_df_temp,ignore_index =True, sort =False)

feat_df

,step type,Gyro0X mean,Gyro0Y mean,Gyro0Z mean,Accel0X mean,Accel0Y mean,Accel0Z mean,Gyro1X mean,Gyro1Y mean,Gyro1Z mean,...,Gyro0Z corr,Accel0X corr,Accel0Y corr,Accel0Z corr,Gyro1X corr,Gyro1Y corr,Gyro1Z corr,Accel1X corr,Accel1Y corr,Accel1Z corr
0,brush_back,-0.015490,0.030227,0.003132,0.065865,0.034069,-0.037633,0.021314,0.022431,-0.014592,...,-0.003146,-0.037383,-0.073456,-0.149435,-0.109363,0.144758,-0.609316,-0.247125,0.158014,0.053246
1,brush_back,0.020174,-0.012924,0.003970,0.008027,-0.002498,-0.081705,0.016634,-0.025017,0.014366,...,0.219924,0.131681,0.200621,-0.150463,0.050964,0.135236,-0.583699,-0.052903,-0.047456,-0.096066
2,brush_back,0.022550,-0.012812,0.004752,0.004388,0.006033,-0.033235,0.023108,-0.022511,0.018694,...,-0.001628,0.154286,-0.046059,-0.159981,0.032151,0.095321,-0.612953,-0.098199,0.132975,-0.027934
3,brush_back,0.029999,-0.013406,0.019847,-0.005353,0.011304,-0.020439,0.018919,-0.017550,0.035601,...,-0.032966,0.127947,0.020145,-0.148668,0.065818,0.123373,-0.623740,-0.093359,0.026251,-0.033035
4,brush_back,0.035406,-0.022083,0.001047,0.006434,-0.001044,-0.010740,0.045474,-0.042480,0.028685,...,0.103957,0.034681,0.121075,-0.137706,0.333161,0.152693,-0.556628,-0.094171,0.074718,0.071208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,step,-0.011680,0.006448,0.027924,0.004178,-0.006852,-0.003926,-0.014199,0.007212,0.017064,...,0.298715,0.099538,0.182652,-0.016028,0.092525,-0.004771,0.116976,0.255168,-0.467539,-0.041184
117,step,-0.020279,0.016623,0.005653,0.007730,-0.021345,-0.005407,-0.021165,0.018701,-0.001929,...,-0.005159,0.130590,0.423095,0.157777,-0.000608,0.287020,-0.147456,0.093544,0.024551,0.374868
118,step,-0.004492,0.025030,0.009241,-0.002509,0.000110,-0.009831,0.000586,0.008983,0.001600,...,0.440372,0.264098,0.440791,0.015101,0.526475,0.398986,0.107826,0.321401,-0.341093,0.074697
119,step,0.008069,0.012146,-0.035170,0.011000,0.008670,-0.009488,0.009482,-0.026933,-0.054340,...,0.212218,0.048654,-0.013285,0.155954,-0.119370,-0.373455,0.208561,0.214176,-0.092198,0.350215


In [6]:
pandas.set_option('display.max_rows', None)
feat_df

,step type,Gyro0X mean,Gyro0Y mean,Gyro0Z mean,Accel0X mean,Accel0Y mean,Accel0Z mean,Gyro1X mean,Gyro1Y mean,Gyro1Z mean,...,Gyro0Z corr,Accel0X corr,Accel0Y corr,Accel0Z corr,Gyro1X corr,Gyro1Y corr,Gyro1Z corr,Accel1X corr,Accel1Y corr,Accel1Z corr
0,brush_back,-0.015490,0.030227,0.003132,0.065865,0.034069,-0.037633,0.021314,0.022431,-0.014592,...,-0.003146,-0.037383,-0.073456,-0.149435,-0.109363,0.144758,-0.609316,-0.247125,0.158014,0.053246
1,brush_back,0.020174,-0.012924,0.003970,0.008027,-0.002498,-0.081705,0.016634,-0.025017,0.014366,...,0.219924,0.131681,0.200621,-0.150463,0.050964,0.135236,-0.583699,-0.052903,-0.047456,-0.096066
2,brush_back,0.022550,-0.012812,0.004752,0.004388,0.006033,-0.033235,0.023108,-0.022511,0.018694,...,-0.001628,0.154286,-0.046059,-0.159981,0.032151,0.095321,-0.612953,-0.098199,0.132975,-0.027934
3,brush_back,0.029999,-0.013406,0.019847,-0.005353,0.011304,-0.020439,0.018919,-0.017550,0.035601,...,-0.032966,0.127947,0.020145,-0.148668,0.065818,0.123373,-0.623740,-0.093359,0.026251,-0.033035
4,brush_back,0.035406,-0.022083,0.001047,0.006434,-0.001044,-0.010740,0.045474,-0.042480,0.028685,...,0.103957,0.034681,0.121075,-0.137706,0.333161,0.152693,-0.556628,-0.094171,0.074718,0.071208
5,brush_back,0.049985,-0.027510,0.020685,-0.017495,0.003801,-0.004606,0.027565,-0.038157,0.061531,...,0.235199,-0.033364,0.087867,-0.140825,0.180569,0.237953,-0.450155,-0.132801,0.045233,-0.011140
6,brush_back,0.017312,-0.023449,0.022354,-0.050029,-0.039332,-0.020180,-0.003083,-0.026713,0.028062,...,0.025740,0.081113,-0.019857,-0.299156,-0.147188,-0.021176,-0.442965,-0.188666,0.058989,-0.120800
7,brush_back,0.028743,-0.027559,-0.001093,-0.046192,-0.011890,-0.015074,0.014812,-0.041796,0.022547,...,-0.213185,0.158913,0.015212,-0.385178,0.002499,-0.313577,-0.344278,-0.076407,0.031047,-0.127797
8,brush_back,0.033332,-0.017685,0.007159,-0.031619,-0.000095,0.009161,0.022146,-0.031000,0.023521,...,-0.280834,0.087250,0.035244,-0.290572,0.039399,-0.199693,-0.387681,-0.126413,0.020908,0.053684
9,brush_back,0.032606,-0.009725,0.013553,0.008339,0.001583,0.012447,0.026723,-0.030021,0.029085,...,-0.139142,0.187279,-0.143348,-0.253539,0.038190,-0.047581,-0.565594,-0.142393,0.128762,0.107127


In [7]:
feat_df.to_csv("etapdance2/step_features_table_filt_truecorr_original_only.txt",index=False)

In [17]:
feat_df[:[:2]]

SyntaxError: invalid syntax (<ipython-input-17-b6b76261ffa3>, line 1)